In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [0]:
%run "../Data_Quality_and_profiling/include/configuration"

In [0]:
%run "../Data_Quality_and_profiling/include/common_functions"

Requirement already satisfied: pyyaml in /databricks/python3/lib/python3.8/site-packages (6.0)
WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

Requirement already satisfied: pretty_html_table in /databricks/python3/lib/python3.8/site-packages (0.9.16)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from pretty_html_table) (1.2.4)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas->pretty_html_table) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas->pretty_html_table) (1.20.1)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas->pretty_html_table) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas->pretty_html_table) (1.15.0)
WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
Requirement already satisfied: openpyxl in /databricks/python3/lib/python3.8/site-packages (3.0.10)
Requirement already satisfied: et-xmlfile in /databricks/python3/lib/python3.8/site-packages (from openpyxl) (1.1.0)
WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
fatal: destination path 'dateinfer' already exists and is not an empty directory.
Processing /databricks/driver/dateinfer
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from pydateinfer==0.4.1) (2020.5)
Building wheels for collected packages: pydateinfer
 Building wheel for pydateinfer (setup.py): started
 Building wheel for pydateinfer (setup.py): finished with status 'done'
 Created wheel for pydateinfer: filename=pydateinfer-0.4.1-py3-none-any.whl size=14397 sha256=1bd1d1d858350d7918ca2e9477989a75621bffc5a7e88488216c4518c34aaf10
 Stored in directory: /tmp/pip-ephem-wheel-cache-clx5f5sr/wheels/b6/f8/d2/6efff5909de6298d820ba0b0e28c430f668df50cc40deacf39
Successfully built pydateinfer
Installing collected packages: pydateinfer
 Attempting uninstall: pydateinfer
 Found existing installation: pydateinfer 0.4.1
 Uninstalling pydateinfer-0.4.1:
 Successfully uninstalled pydateinfer-0.4.1
Successfully installed pydateinfer-0.4.1
WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
omd_sdf =  spark.read.options(delimiter=',') \
           .option("header" , True) \
           .csv(f'{dq_raw_folder_path}France_Final_v5.csv')


## Driver Code

In [0]:
df = preprocess_data(omd_sdf, numeric_cols = ['Net Spend (Local)','Impressions','Reach', 'Clicks','Valued Views','Spot Length (sec)','GRPs','Adjusted GRPs','TRPs','Impacts (000s)','Engagements'])
df = auto_date_parser(parse_df=df,date_cols = ['Campaign Start Date','Campaign End Date','Week'], final_date_fmt = "yyyy-MM-dd")

df = df.withColumn("Net Spend (Local)", df["Net Spend (Local)"].cast(IntegerType()).alias("Net Spend (Local)"))

#Dropping rows with all na 
df = df.na.drop("all")



In [0]:
data_quality_cols_regex = {}
date_cols = [col for col in df.columns if 'Date' in col or 'Week' in col]
date_regex = '[0-9]{4}-(((0[13578]|(10|12))-(0[1-9]|[1-2][0-9]|3[0-1]))|(02-(0[1-9]|[1-2][0-9]))|((0[469]|11)-(0[1-9]|[1-2][0-9]|30)))$'
data_quality_cols_regex =  dict.fromkeys(date_cols,date_regex)


# Constants
numeric_cols = ['Net Spend (Local)','Impressions','Reach']
aggregate_cols = [ 'Brand Generation','Product','Week','Year-Week']
result_limit = 100


In [0]:
### 1. NULL Checks
resultdf = get_null_perc(spark, df, null_cols= df.columns)
print("NULL/Empty Percentage for Columns")
resultdf.show(result_limit,False)

NULL/Empty Percentage for Columns
+---------------------------+--------------+
Column |NullPercentage|
+---------------------------+--------------+
Country |0.00% |
Category |100.00% |
Agency |0.00% |
Brand Generation |0.00% |
Product |0.00% |
Campaign |0.00% |
CampaignID |35.09% |
Sub Campaign |17.64% |
Campaign Start Date |0.00% |
Campaign End Date |0.00% |
Planning Target |0.98% |
Media Mix |0.00% |
Channel |0.00% |
Channel Detail |14.57% |
Media Type |0.86% |
Digital Platform |37.17% |
Buying Type |37.05% |
Device |38.09% |
Format |33.62% |
Net Spend (Local) |0.00% |
Currency ISO Code |0.00% |
Spot Length (sec) |5.14% |
GRPs |66.32% |
Adjusted GRPs |100.00% |
TRPs |71.22% |
Admissions |99.02% |
Impacts (000s) |98.96% |
Number/Length of Executions|97.37% |
Impressions |34.29% |
Clicks |38.27% |
Engagements |57.38% |
Valued Views |37.78% |
Reach |37.60% |
Frequency |36.56% |
Comments |83.34% |
OOH Format Category |99.94% |
Week |0.00% |
Facebook Objective |95.10% |
Facebook Buying Type |95.16% |
YouTube Format |95.77% |
Year |0.00% |
+---------------------------+--------------+

In [0]:
###2. Summary, Average, Standard Deviation, Percentiles for Numeric Columns
resultdf = get_summary_numeric(df, numeric_cols)
print("Summary for Numeric Columns")
resultdf.show(result_limit, False)

Summary for Numeric Columns
+-------+-----------------+------------------+------------------+
summary|Net Spend (Local)|Impressions |Reach |
+-------+-----------------+------------------+------------------+
count |1633 |1073 |1019 |
mean |29025.20453153705|3021233.5582478843|865112.060255152 |
stddev |40623.52177657263|4516790.881784353 |1974449.9103545486|
min |0 |10 |0 |
25% |1366 |109504.0 |48.0 |
50% |13125 |576330.0 |1621.0 |
75% |39275 |4921598.0 |350199.0 |
max |471799 |9960894 |9998411 |
+-------+-----------------+------------------+------------------+

In [0]:
###3. Distinct Count
print("Distinct Counts for Aggregate Columns")
resultdf = get_distinct_counts(spark, df, aggregate_cols)
resultdf.show(result_limit, False)

Distinct Counts for Aggregate Columns
+----------------+-------------+
Column |DistinctCount|
+----------------+-------------+
Brand Generation|17 |
Product |36 |
Week |132 |
+----------------+-------------+

In [0]:
###4. Distribution Count
print("Distribution Count for Aggregate Columns")
result = get_distribution_counts(spark, df, aggregate_cols)
for i in result:
	print("======== Distribution for - " + i.columns[0] + " ========")
	i.show(result_limit, False)

Distribution Count for Aggregate Columns
======== Distribution for - Brand Generation ========
+----------------+-----+
Brand Generation|count|
+----------------+-----+
MARVINS |555 |
SOMRAS |463 |
PREFECT |131 |
WALNUTS |120 |
BENIGN |77 |
ICHOR |65 |
FRACTAL |54 |
TSC-TM |41 |
TSCPOWER |36 |
REMEDY |28 |
Remedy |18 |
Somras |15 |
DAWNS |12 |
TscPower |7 |
Ichor |4 |
MULTI-BRAND |4 |
Benign |3 |
+----------------+-----+

======== Distribution for - Product ========
+-------------------------+-----+
Product |count|
+-------------------------+-----+
MARVINS-CORE |358 |
SOMRAS-PLASTICITY-CORE |243 |
SOMRAS-PLASTICITY-GREEN |129 |
SOMRAS-MULTI |90 |
TSC-POWER |84 |
PREFECT-CORE |83 |
BENIGN-CORE |53 |
MARVINS CHEFSPECIAL |52 |
WALNUTS-CORE |49 |
MARVINS 6S |43 |
MARVINS-OTHER |33 |
WALNUTS-6SS-TRUMPETS |33 |
MARVINS-TRUMPETS |31 |
FRACTAL-CORE |31 |
REMEDY |28 |
PREFECT |28 |
VEGSTEW-ORIGINAL |26 |
ICHOR-MELONSODA |25 |
BENIGN |24 |
FRACTAL |23 |
WALNUTS ELATED BITTERNESS|23 |
CORE |21 |
PREFECT NODY |20 |
WALNUTS CORE |16 |
ICHOR |14 |
DAWNS-CORE |12 |
MARVINS-EASTBLUE |12 |
SOMRAS-GREEN |10 |
6SS-CORE |8 |
WALNUTS-COOKED-ALRIGHT |7 |
SOMRAS-CORE |6 |
MARVINS EDGE IIT |5 |
MULTI-PRODUCT |4 |
Ichor |4 |
MARVINS CORE |4 |
MARVINS EDGE |1 |
+-------------------------+-----+

======== Distribution for - Week ========
+----------+-----+
Week |count|
+----------+-----+
2021-06-21|65 |
2019-07-29|62 |
2019-08-26|54 |
2021-12-13|36 |
2019-08-05|35 |
2020-08-03|32 |
2019-08-19|32 |
2020-08-10|30 |
2020-07-27|30 |
2019-08-12|29 |
2020-07-20|28 |
2020-05-18|27 |
2020-06-01|26 |
2019-06-17|25 |
2021-12-20|25 |
2019-06-10|25 |
2020-05-25|24 |
2021-06-28|22 |
2020-12-14|20 |
2020-12-07|20 |
2020-07-13|20 |
2020-08-31|20 |
2021-06-14|19 |
2020-12-28|19 |
2020-12-21|19 |
2021-12-06|19 |
2019-06-24|19 |
2021-07-05|18 |
2019-07-22|17 |
2019-09-02|17 |
2019-06-03|17 |
2020-06-15|16 |
2021-07-12|16 |
2019-07-15|15 |
2020-08-24|15 |
2020-09-07|15 |
2021-05-24|15 |
2020-08-17|15 |
2021-05-17|14 |
2019-07-01|14 |
2021-05-31|14 |
2020-06-08|14 |
2021-05-10|13 |
2019-07-08|13 |
2019-09-16|13 |
2020-06-29|13 |
2020-06-22|13 |
2020-10-05|13 |
2021-06-07|13 |
2021-11-29|13 |
2021-08-09|12 |
2021-05-03|12 |
2021-07-19|12 |
2019-09-09|11 |
2020-04-06|11 |
2021-07-26|11 |
2021-08-02|11 |
2019-10-21|10 |
2020-09-28|10 |
2021-08-16|10 |
2020-01-27|10 |
2020-07-06|10 |
2020-10-12|9 |
2021-11-22|9 |
2021-12-27|9 |
2021-04-26|9 |
2020-10-19|9 |
2020-03-23|9 |
2021-04-12|9 |
2019-10-28|9 |
2020-10-26|9 |
2021-09-06|8 |
2021-04-19|8 |
2020-02-03|8 |
2019-09-23|8 |
2020-09-21|8 |
2021-08-23|8 |
2019-05-27|7 |
2021-11-08|7 |
2019-09-30|7 |
2020-02-24|7 |
2020-05-11|7 |
2021-04-05|7 |
2019-11-04|7 |
2020-02-17|7 |
2020-09-14|7 |
2020-04-13|7 |
2020-03-09|7 |
2020-03-30|7 |
2020-04-20|7 |
2021-08-30|6 |
2019-04-29|6 |
2019-05-13|6 |
2019-05-20|6 |
2021-11-01|6 |
2021-11-15|6 |
2019-04-22|5 |
2019-04-01|5 |
2020-03-16|5 |
2021-03-01|5 |
+----------+-----+
only showing top 100 rows

In [0]:
###5. Data Quality
print("Data Quality Issue Percentage for Columns")
resultdf = get_mismatch_perc(spark, df, data_quality_cols_regex)
resultdf.show(result_limit, False)

Data Quality Issue Percentage for Columns
+-------------------+------------------+
Column |MismatchPercentage|
+-------------------+------------------+
Campaign Start Date|0.0% |
Campaign End Date |0.0% |
Week |0.0% |
+-------------------+------------------+

In [0]:
null_check_columns = ['CampaignID', 'Sub Campaign', 'Digital Platform']

err_df = err_df_null_rule(threshold_per = 10, null_check_columns = null_check_columns, df = df.toPandas())

In [0]:
dest_path = generate_report(raw_df = df.toPandas(),err_df = err_df,destination_path=dbfs_processed_path)
print(f'DQ file is generated in the location {dest_path}')

DQ file is generated in the location /dbfs/FileStore/DQ/DQ_Report.xlsx

In [0]:
stage_data_to_s3()

file loaded to s3 path demo-s3-stage/DQ_Report.xlsx

### Reconciliation TOOL

In [0]:
media_df = df
media_mapping_df =pd.read_excel(f'{dbfs_mapping_path}ROI_mappings_FR.xlsx',sheet_name = 'Media_Mappings')
media_mapping_df = spark.createDataFrame(media_mapping_df.astype(str))
media_mapping_campaign_df = pd.read_excel(f'{dbfs_mapping_path}ROI_mappings_FR.xlsx', sheet_name = 'TV Campaign Mapping')
media_mapping_campaign_df = spark.createDataFrame(media_mapping_campaign_df.astype(str))

In [0]:
reconcilliation_tool(config_string, get_fuzzy_match = True, tol_val = 1.0)

Matching between Product and MEDIA_BRAND

Number of Unique Values in Product : 36

Number of Unique Values in MEDIA_BRAND : 39

Product,MEDIA_BRAND,_merge
6SS-CORE,6SS-CORE,both
BENIGN,null,Raw Media Data
BENIGN-CORE,BENIGN-CORE,both
BENIGN-CORE,BENIGN-CORE,both
null,BENIGN-ESSENTIALS,Media Mapping Data
CORE,null,Raw Media Data
DAWNS-CORE,DAWNS-CORE,both
FRACTAL,FRACTAL,both
FRACTAL-CORE,FRACTAL-CORE,both
null,FRED-CORE,Media Mapping Data


Match Result Percentage : 64.58

...

Number of Unique Values in GROUP_NAME : 4

Brand Generation,GROUP_NAME,_merge
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data
null,0,Media Mapping Data


Match Result Percentage : 10.53

Fuzzy Match between Brand Generation and GROUP_NAME

Brand Generation,GROUP_NAME
Somras,Somras
SOMRAS,Somras


In [0]:
reconcilliation_tool(config_string1, get_fuzzy_match = True, tol_val = 1.0)

Matching between Campaign and Raw TV Campaign

Number of Unique Values in Campaign : 69

Number of Unique Values in Raw TV Campaign : 29

Campaign,Raw TV Campaign,_merge
6S,6S,both
6S,6S,both
6S,6S,both
6S,6S,both
Benign fin d'annÃ©e,null,Raw Media Data
Burning Desires,null,Raw Media Data
CELEBRE,null,Raw Media Data
CHEFSPECIAL,CHEFSPECIAL,both
CITRUS,null,Raw Media Data
CORE,CORE,both


Match Result Percentage : 36.47

Fuzzy Match between Campaign and Raw TV Campaign

Campaign,Raw TV Campaign
Equity Power,Equity Power
Power,Power
NA,6S
Citrus,Citrus
6S,6S
GRIN HOLEE,GRIN HOLEE
Grin,GREEN
Grin,GRIN
Eastblue,Eastblue
PREFECT,Prefect


In [0]:
dbutils.fs.mounts()

Out[1]: [MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/mnt/azdatabrickscourse/raw', source='abfss://raw@azdatabrickscourse.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/azdatabrickscourse/processed', source='abfss://processed@azdatabrickscourse.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='')]

In [0]:
dbutils.fs.ls('/mnt/azdatabrickscourse/raw/DQ_Raw/')

Out[3]: [FileInfo(path='dbfs:/mnt/azdatabrickscourse/raw/DQ_Raw/France_Final_v5.csv', name='France_Final_v5.csv', size=473170, modificationTime=1663739062000),
 FileInfo(path='dbfs:/mnt/azdatabrickscourse/raw/DQ_Raw/GB_Ext_OMD_Media.csv', name='GB_Ext_OMD_Media.csv', size=573719, modificationTime=1663506943000),
 FileInfo(path='dbfs:/mnt/azdatabrickscourse/raw/DQ_Raw/ROI_mappings_FR.xlsx', name='ROI_mappings_FR.xlsx', size=190788, modificationTime=1663739026000)]

In [0]:
%sh
rm -r /dbfs/mnt/azdatabrickscourse/raw/DQ_Raw/